## 2 Layer MLP w/InfiMNIST

500 epochs of RMSProp


We can only runn 75 -150 epopchs of ALignMNIST

Here, we can go 'forever' ... upto 16K epochs ... so lets try ... starting with 500

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
%run augmentmnist.py

In [3]:
num_epochs = 500
batch_size = 125

learning_rate = 0.001

In [4]:
# Network Parameters
n_hidden_1 = 1000 # 1st layer num features
n_hidden_2 = 500 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

std_0 = 1.0/np.sqrt(n_input)
std_h1 = 1.0/np.sqrt(n_hidden_1)
std_h2 = 1.0/np.sqrt(n_hidden_2)

logfile = "2-layer-mlp-infimnist"

In [5]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

xt = tf.placeholder("float", [None, n_input])
yt = tf.placeholder("float", [None, n_classes])

In [6]:
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    return tf.matmul(layer_2, weights['out']) + biases['out']

In [7]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=std_0)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=std_h1)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=std_h2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], stddev=0.1)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2], stddev=0.01)),
    'out': tf.Variable(tf.random_normal([n_classes], stddev=0.001))
}

In [8]:
mlp = multilayer_perceptron(x, weights, biases )
mlp_test = multilayer_perceptron(xt, weights, biases )

In [9]:
cost =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(mlp, y)) 
gs = tf.get_variable("global_step",[],trainable=False,initializer=tf.constant_initializer(0))
lr = tf.constant(learning_rate) 

In [10]:
train_op = tf.contrib.layers.optimize_loss(cost, global_step=gs, learning_rate=lr,optimizer="RMSProp")

In [11]:
#with tf.name_scope("training accuracy"):
pred = tf.equal(tf.argmax(mlp, 1), tf.argmax(y, 1)) # Count correct predictions
train_acc_op = tf.reduce_mean(tf.cast(pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("training accuracy", train_acc_op)

<tf.Tensor 'ScalarSummary_2:0' shape=() dtype=string>

In [12]:
test_pred = tf.equal(tf.argmax(mlp_test, 1), tf.argmax(yt, 1)) # Count correct predictions
test_acc_op = tf.reduce_mean(tf.cast(test_pred, "float"))  # Cast boolean to float to average
tf.scalar_summary("test 0 accuracy", test_acc_op)

<tf.Tensor 'ScalarSummary_3:0' shape=() dtype=string>

In [13]:
!rm -rf ./logs/{logfile}
!ls logs

2-layer-mlp-alignmnist 5-layer-mlp-alignmnist


### Original MNIST Data

In [14]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True)
trX_0, trY_0 = mnist.train.images, mnist.train.labels
teX_0, teY_0 = mnist.test.images, mnist.test.labels

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


### InfiMNIST run

In [15]:
infiminst = InfiMNIST()

In [17]:
test_accuracies = []
train_accuracies = []
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs/{logfile}'
    writer = tf.train.SummaryWriter("./logs/{0}".format(logfile), sess.graph) # for 0.8
    merged = tf.merge_all_summaries()
 
    tf.initialize_all_variables().run()

    i = -1
    for epoch in range(num_epochs):
        
        i+=1
        log=((i%10)==0)
            
        trX, trY = infiminst.next_epoch(log)
        
        # permute / shuffle the images
        pids = np.arange(trY.shape[0])
        np.random.shuffle(pids)
        trX, trY = trX[pids], trY[pids]
        
        for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX), batch_size)):
            sess.run(train_op, feed_dict={x: trX[start:end], y: trY[start:end]})
                 
        summary, trn_acc, tst_acc = sess.run([merged, train_acc_op, test_acc_op], feed_dict={x: trX, y: trY, xt: teX_0, yt: teY_0})
        writer.add_summary(summary, epoch)  
        
        if (log):
            print(i,epoch, trn_acc, tst_acc)         
        
        train_accuracies.append(trn_acc)
        test_accuracies.append(tst_acc)

        writer.flush()
        

creating infimnist pat files 10000 - 69999
updating 80000 - 139999


//anaconda/lib/python2.7/gzip.py:275: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
<string>:59: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


(0, 0, 0.96815002, 0.96100003)
creating infimnist pat files 710000 - 769999
updating 780000 - 839999
(10, 10, 0.98858333, 0.98439997)
creating infimnist pat files 1410000 - 1469999
updating 1480000 - 1539999
(20, 20, 0.99343336, 0.98680001)
creating infimnist pat files 2110000 - 2169999
updating 2180000 - 2239999
(30, 30, 0.99641669, 0.98720002)
creating infimnist pat files 2810000 - 2869999
updating 2880000 - 2939999
(40, 40, 0.99554998, 0.98650002)
creating infimnist pat files 3510000 - 3569999
updating 3580000 - 3639999
(50, 50, 0.99475002, 0.98650002)
creating infimnist pat files 4210000 - 4269999
updating 4280000 - 4339999
(60, 60, 0.9964667, 0.98479998)
creating infimnist pat files 4910000 - 4969999
updating 4980000 - 5039999
(70, 70, 0.99615002, 0.98659998)
creating infimnist pat files 5610000 - 5669999
updating 5680000 - 5739999
(80, 80, 0.99776667, 0.98860002)
creating infimnist pat files 6310000 - 6369999
updating 6380000 - 6439999
(90, 90, 0.99676669, 0.98760003)
creating in

In [18]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
plt.plot(train_accuracies)
plt.plot(test_accuracies)

In [ ]:
plt.show()